In [ ]:
import torch
import torch.autograd as autograd
from torch import Tensor
import torch.nn as nn
import torch.optim as optim

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import time
import scipy.io

import matplotlib.pyplot as plt

torch.set_default_dtype(torch.float)

torch.manual_seed(1234)
np.random.seed(1234)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
data = pd.read_csv('bio.csv', sep=',')
data = data.replace(np.nan, 0.)

In [ ]:
t = data['t'].to_numpy().reshape(-1,1)
cB = data['cB'].to_numpy().reshape(-1,1)
cTG = data['cTG'].to_numpy().reshape(-1,1)
cDG = data['cDG'].to_numpy().reshape(-1,1)
cMG = data['cMG'].to_numpy().reshape(-1,1)
cG = data['cG'].to_numpy().reshape(-1,1)
c = np.concatenate((cB, cTG, cDG, cMG, cG), axis=1)
c

In [ ]:
total_points = 4

t_train = np.linspace(t[0], t[-1], total_points)

idx = []
for ti in t:
    idx.append(np.where(t_train.reshape(-1,1) == ti)[0][0])
idx = np.array(idx)
idx

In [ ]:
c_train = np.zeros((total_points,5))
c_train[idx,0] = cB.flatten()
c_train[idx,1] = cTG.flatten()
c_train[idx,2] = cDG.flatten()
c_train[idx,3] = cMG.flatten()
c_train[idx,-1] = cG.flatten()
c_train

In [ ]:
t_train = torch.from_numpy(t_train).float().to(device)
c_train = torch.from_numpy(c_train).float().to(device)

In [ ]:
k1 = 1.
k2 = 1.
k3 = 1.
k4 = 1.
k5 = 1.
k6 = 1.

layers = np.array([1,10,10,10,5])

f_hat = torch.zeros(t_train.shape[0],1).to(device)

In [ ]:
class DNN(nn.Module):
     
    def __init__(self, layers):
        super().__init__()
        
        self.activation = nn.Tanh()

        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])

        for i in range(len(layers)-1):
            
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            
            nn.init.zeros_(self.linears[i].bias.data)
            
    def forward(self, x):
              
        if torch.is_tensor(x) != True:
            x = torch.from_numpy(x)
        
        a = x.float()
        
        for i in range(len(layers)-2):
            
            z = self.linears[i](a)
                        
            a = self.activation(z)
            
        a = self.linears[-1](a)
        
        return a

In [ ]:
class FCN():
    
    def __init__(self, layers):
        self.dnn = DNN(layers).to(device)
        self.loss_function = nn.MSELoss(reduction = 'mean')

        self.iter = 0

        self.k1 = torch.tensor([k1], requires_grad=True).float().to(device)
        self.k2 = torch.tensor([k2], requires_grad=True).float().to(device)
        self.k3 = torch.tensor([k3], requires_grad=True).float().to(device)
        self.k4 = torch.tensor([k4], requires_grad=True).float().to(device)
        self.k5 = torch.tensor([k5], requires_grad=True).float().to(device)
        self.k6 = torch.tensor([k6], requires_grad=True).float().to(device)

        self.k1 = nn.Parameter(self.k1)
        self.k2 = nn.Parameter(self.k2)
        self.k3 = nn.Parameter(self.k3)
        self.k4 = nn.Parameter(self.k4)
        self.k5 = nn.Parameter(self.k5)
        self.k6 = nn.Parameter(self.k6)

        self.dnn = DNN(layers).to(device)
        
        self.dnn.register_parameter('k1', self.k1)
        self.dnn.register_parameter('k2', self.k2)
        self.dnn.register_parameter('k3', self.k3)
        self.dnn.register_parameter('k4', self.k4)
        self.dnn.register_parameter('k5', self.k5)
        self.dnn.register_parameter('k6', self.k6)
        
    def loss_data(self, x, y):
        
        #cum_loss = []
        #for i in range(5):
        #    loss_c = self.loss_function(self.dnn(x)[:,i], y[:,i].reshape(-1,1))
        #    if all(y[:,i] == 0):
        #        loss_c.zero_()
        #    cum_loss.append(loss_c)
            
        loss_u = self.loss_function(self.dnn(x), y)
        
        return loss_u
    
    def dc_dt(self, x, i):
        
        g = x.clone()
        
        g.requires_grad = True
        
        conc = self.dnn(g)
        
        grad_c = autograd.grad(conc[:,i].reshape(-1,1), g, torch.ones(x.shape[0], 1).to(device), retain_graph=True, create_graph=True)[0]
        
        return grad_c
    
    def c_t(self, x, i):
        
        conc = self.dnn(x)
        
        eval_c = conc[:,i].reshape(-1,1)
        
        return eval_c
    
    def loss_ode(self, x, y):
                 
        loss_f = self.loss_function(self.dc_dt(x, 1) + self.k1*self.c_t(x, 1) - self.k2*self.c_t(x, 2)*self.c_t(x, 0), f_hat) + \
                 self.loss_function(self.dc_dt(x, 2) - self.k1*self.c_t(x, 1) + self.k2*self.c_t(x, 2)*self.c_t(x, 0) \
                                                     + self.k3*self.c_t(x, 2) - self.k4*self.c_t(x, 3)*self.c_t(x, 0), f_hat) + \
                 self.loss_function(self.dc_dt(x, 3) - self.k3*self.c_t(x, 2) + self.k4*self.c_t(x, 3)*self.c_t(x, 0) \
                                                     + self.k5*self.c_t(x, 3) - self.k6*self.c_t(x, 4)*self.c_t(x, 0), f_hat) + \
                 self.loss_function(self.dc_dt(x, 4) - self.k5*self.c_t(x, 3) + self.k6*self.c_t(x, 4)*self.c_t(x, 0), f_hat) + \
                 self.loss_function(self.dc_dt(x, 0) - self.k1*self.c_t(x, 1) + self.k2*self.c_t(x, 2)*self.c_t(x, 0) \
                                                     - self.k3*self.c_t(x, 2) + self.k4*self.c_t(x, 3)*self.c_t(x, 0) \
                                                     - self.k5*self.c_t(x, 3) + self.k6*self.c_t(x, 4)*self.c_t(x, 0), f_hat)
        
        return loss_f
    
    def closure(self):
        
        optimizer.zero_grad()
        
        loss_u = self.loss_data(t_train, c_train)
        loss_f = self.loss_ode(t_train, c_train)
        
        loss = 100*loss_u + loss_f
        
        (100*loss_u + loss_f).backward()
                
        self.iter += 1
        
        return loss

In [ ]:
PINN = FCN(layers)

print(PINN.dnn)

In [ ]:
params = list(PINN.dnn.parameters())

optimizer = torch.optim.Adam(params, lr=0.001)

while PINN.iter < 1000:
    optimizer.step(PINN.closure)
    for p in PINN.dnn.parameters():
        p.data.clamp_(min=0)

print(PINN.dnn(t_train))

In [ ]:
print(PINN.loss_data(t_train, c_train))
print(PINN.loss_ode(t_train, c_train))

In [ ]:
def EDO(y, prm):
    
    k1 = prm[0]
    k2 = prm[1]
    k3 = prm[2]
    k4 = prm[3]
    k5 = prm[4]
    k6 = prm[5]
    
    f = np.zeros(5)
    
    f[1] = - k1*y[1] + k2*y[2]*y[0]
    f[2] = + k1*y[1] - k2*y[2]*y[0] - k3*y[2] + k4*y[3]*y[0]
    f[3] = + k3*y[2] - k4*y[3]*y[0] - k5*y[3] + k6*y[4]*y[0]
    f[4] = + k5*y[3] - k6*y[4]*y[0]
    f[0] = + k1*y[1] - k2*y[2]*y[0] + k3*y[2] - k4*y[3]*y[0] + k5*y[3] - k6*y[4]*y[0]
    
    return f

def euler_explicite(y0, dt, tf, prm):
    
    mat_y = np.array([y0])
    
    t = np.array([0])
    while t[-1] < tf:
        y = y0 + dt * EDO(y0, prm)
        
        mat_y = np.append(mat_y, [y], axis=0)
        t = np.append(t, t[-1]+dt)
        
        y0 = np.copy(y)
    
    return t, mat_y

prm = []        
for i in range(6):
    prm.append(params[:6][i][0].cpu().detach().numpy())
    
t_euler, y_euler = euler_explicite(np.array([0,0.540121748,0.057018273,0,0]), 0.001, 6, prm)

In [ ]:
plt.plot(t_euler, y_euler[:,0])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train).detach().numpy()[:,0])
plt.plot(t, cB, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,1])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train).detach().numpy()[:,1])
plt.plot(t, cTG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,2])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train).detach().numpy()[:,2])
plt.plot(t, cDG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,3])
plt.plot(t_train.detach().numpy(), PINN.dnn(t_train).detach().numpy()[:,3])
plt.plot(t, cMG, 'o')
plt.show()

In [ ]:
plt.plot(t_euler, y_euler[:,4])
plt.plot(t, cG, 'o')
plt.show()

In [ ]:
prm